In [79]:
import importlib
import SpotifyAPIClient
import pprint
import csv
import os
importlib.reload(SpotifyAPIClient)
from SpotifyAPIClient import SpotifyAPI
import pandas as pd

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils


# uncomment once you paste your mypytable.py into mysklearn package
# import mysklearn.mypytable
# importlib.reload(mysklearn.mypytable)
# from mysklearn.mypytable import MyPyTable 

client_id = 'd7a2e6f4a8434550baa5eda073f0a6a3'
client_secret = 'def46d47ba584e378b7667645666a468'
playlist_id = '7L736vCRhBe5EapwwkutUl'

In [80]:
spotify = SpotifyAPI(client_id, client_secret)
response = spotify.get_playlist(playlist_id)


In [81]:
track_ids = spotify.parse_for_track_id(response)
#print("track_ids: ", track_ids)
print()
print()
header = ["Track Name", "Track ID", "Danceability", "Energy", "valence"] 
tracks_feature_list = []

did_once = False
features_header = []
features_header.append("song_id")
features_header.append("song_name")
songs_vals_list = [] #[song_id, song_name, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, type, id, uri, track_href, analysis_url, duration_ms, time_signature ]

for name, id in track_ids.items():
    #print("name: ", name, " ----- id: ", id)
    song = []
    song.append(id)
    song.append(name)

    track_feat_list = []
    response = spotify.get_track_audio_features(id)
    #print("response: ", response)

    for k, v in response.items():
        #print("     k: ", k, "      v: ", v)
        if not did_once:
            features_header.append(k)
        song.append(v)
    did_once = True
    songs_vals_list.append(song)

    track_feat_list.append(name)
    track_feat_list.append(id)
    track_feat_list.append(round(response['danceability'] * 100, 2))
    track_feat_list.append(round(response['energy'] * 100, 2))
    # track_feat_list.append(response['tempo'])
    track_feat_list.append(round(response['valence'] * 100, 2))
    tracks_feature_list.append(track_feat_list)

#print("features_header: ", features_header)
#print("songs_vals_list: ", songs_vals_list)
#myutils.table_val_print(features_header, songs_vals_list)
#myutils.print_tabulate(songs_vals_list, features_header)
myutils.print_tabulate(tracks_feature_list, header)


    



====================================================  ======================  ==============  ========  =========
Track Name                                            Track ID                  Danceability    Energy    valence
====================================================  ======================  ==============  ========  =========
Are We There Yet                                      39LRXm8xZ3DP2W37sl1NDj            71.1      58.1       62.1
More Than You Know                                    4q8PHoRsPUB52LFylX8Ulz            64.5      74.1       53.4
Sweet Nothing (feat. Florence Welch)                  24LS4lQShWyixJ0ZrJXfJ5            57.3      92.9       58.2
Wish You Were Mine - Radio Edit                       412luShbmlgqqgYFStIB1s            65.4      83.2       40.7
Personal                                              5kflnIKOUU6aLXkVXmJ6iK            52.2      69.7       44.7
Sit Next to Me                                        14aqr8yZODpQyuqhSxLMTV          

In [82]:
file_name = os.path.join('data', 'key-track-audio-features.txt')
with open(file_name, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(header)
    csv_writer.writerows(tracks_feature_list)

file_name2 = os.path.join('data', 'track-audio-features_all.txt')
with open(file_name2, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(features_header)
    csv_writer.writerows(songs_vals_list)